# ДЗ 2. Проверка гипотез

Имя, Фамилия: Шашлова Елизавета

группа: 224 группа

**Оценка(для проверяющего):** 32 из 32

**Дедлайн:** 24 ноября 23:59 (решение нужно сдать в энитаск)

# Введение. Несколько слов о шахматах

В этом домашнем задании вам будет представлен набор данных связанных с шахматами. Напомним, что шахматная партия может заканчиваться одним из трёх исходов:
- Победа белых (записывается как 1-0)
- Победа чёрных (записывается как 0-1)
- Ничейный исход (записывается как 1/2-1/2)

Также, в соревновательных шахматах есть временной контроль: партии делятся на несколько категорий, в зависимости от того, сколько времени оппонненты имеют на размышления:
- Блиц (до 10 минут на все ходы, возможно с добавлением времени после каждого хода)
- Рапид (от 10 до 30 минут на все ходы, возможно с добавлением времени после каждого хода)
- Классические или медленные шахматы (от 30 минут на все ходы, часто с добавлением времени после 40 и 60 ходов)

Для того чтобы как-то классифицировать силу игроков, была введена рейтинговая система ELO, на основании которой игрокам присуждается ранг (например международный мастер или гроссмейстер) и формируются турниры. Отметим что рейтинг ELO меняется, в зависимости от того, в каком формате временного контроля игрок участвует (т.е. у одного человека разный рейтинг для блица, рапида и классических шахмат).

В этом датасете собраны основные данные об шахматных партиях, сыгранными за период с 1980 по 2021 года.
- Представлены партии всех трех форматов, формат партии указан в поле ``control``
- Исход партии представлен в поле ``result``
- Рейтинги оппонентов записаны в полях ``elo_white`` и ``elo_black`` для рейтинга играющих за белых и за черных соответственно.  
    - Поле ``elo_average`` содержит среднее значение ``elo_white`` и ``elo_black`` -- _усредненный рейтинг (оппонентов)_.
    - Поле ``elo_difference`` содержит в себе значение ``elo_white - elo_black``.
- Поле ``opening`` содержит одно из пяти значений ``'A', 'B', 'C', 'D', 'E'``, которое кодирует информацию о нескольких первых ходах в партии (см. задачу 3 для деталей).
- В поле ``length`` записано число сделанных ходов в партии. Обратите внимание, что ход -- это движение одной фигуры со стороны белых **и** движение одной фигуры со стороны чёрных, т.е. за один ход и белые и черные совершают по действию.
- Поля ``date`` и ``site`` содержат информацию о том, в каком году была сыграна партия и место её проведения соответственно.

## Обзор данных (2 балла)
В этой секции вам предлагается познакомиться с данными. Посмотрите на три разных формата временного контроля. Сравните как в каждом из них
- Распределены исходы партий (победа белых/победа чёрных/ничья)
- Распределены усреднённые рейтинги оппонентов партий
- Распределены длительности партий

**Бонус (max 3 дополнительных балла)** При желании, можно также посмотреть другие вещи --- например как менялось количество партий по декадам или пятилеткам. Попробуйте обнаружить и _объяснить_ интересные и статистически значимые феномены.  

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.precision', 5)
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns

#add your imports if necessary

: 

In [ ]:
games_df = pd.read_csv("/content/games.csv")
games_df.head()

: 

In [ ]:
games_df.info(verbose = True)

: 

In [ ]:
blitz_games = games_df[games_df['control'] == 'blitz']
rapid_games = games_df[games_df['control'] == 'rapid']
slow_games = games_df[games_df['control'] == 'slow']

: 

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(7, 10))

blitz_games['result'].value_counts(normalize=True).plot.bar(ax=axes[0], title='Распределение результатов партий в блиц-играх', color='skyblue')
axes[0].set_ylabel('Частота')

rapid_games['result'].value_counts(normalize=True).plot.bar(ax=axes[1], title='Распределение результатов партий в рапид-играх', color='orange')
axes[1].set_ylabel('Частота')

slow_games['result'].value_counts(normalize=True).plot.bar(ax=axes[2], title='Распределение результатов партий в медленных играх', color='green')
axes[2].set_ylabel('Частота')

plt.tight_layout()
plt.show()

: 

Видим, что

1) Во всех трех временных контролях доминирует победа белых

2) Победа черных имеет примерно одинаковое распредение на всех терх временных контролях

3)Количество партий, которые закончились ничьей, становится больше с увеличением продолжительности игры. Видимо, из-за ограниченного времени игроки чаще совершают ошибки, что снижает вероятность ничьих



Построим для каждого из контролей распределение усредненного рейтинга партий и длин партий.

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(7, 10))

blitz_games['elo_average'].plot.hist(ax=axes[0], bins=20, title='Распределение усредненного рейтига в блиц-играх', color='skyblue')
axes[0].set_xlabel('Усредненный рейтинг')
axes[0].set_ylabel('Частота')

rapid_games['elo_average'].plot.hist(ax=axes[1], bins=20, title='Распределение усредненного рейтига в рапид-играх', color='orange')
axes[1].set_xlabel('Усредненный рейтинг')
axes[1].set_ylabel('Частота')

slow_games['elo_average'].plot.hist(ax=axes[2], bins=20, title='Распределение усредненного рейтига в медленных играх', color='green')
axes[2].set_xlabel('Усредненный рейтинг')
axes[2].set_ylabel('Частота')

plt.tight_layout()
plt.show()

: 

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(7, 10))

blitz_games['length'].plot.hist(ax=axes[0], bins=20, title='Распределение длительности партий в блиц-играх', color='skyblue')
axes[0].set_xlabel('Длительность партии')
axes[0].set_ylabel('Частота')

rapid_games['length'].plot.hist(ax=axes[1], bins=20, title='Распределение длительности партий в рапид-играх', color='orange')
axes[1].set_xlabel('Длительность партии')
axes[1].set_ylabel('Частота')

slow_games['length'].plot.hist(ax=axes[2], bins=20, title='Распределение длительности партий в медленных играх', color='green')
axes[2].set_xlabel('Длительность партии')
axes[2].set_ylabel('Частота')

plt.tight_layout()
plt.show()

: 

# Задачи

## Разминка. Проверка гипотез о средних (5 баллов)

### Взгляд в прошлое. Рейтинг игр в Праге. (2 балла)
Пусть $X_1, \ldots, X_n \overset{\text{i.i.d}}{\sim} X$ --- выборка из некоторого распределения. Вспомните как строится асимптотический доверительный интервал для $\mathbb{E}[X]$, и, используя асимптотический z-test проверьте на уровне значимости $\alpha = 0.05$ гипотезу о том что математическое ожидание рейтинга партии равно 2389:
$$
    H_0 : \mathbb{E}[X] = 2389 \\
    H_1 : \mathbb{E}[X] \neq 2389
$$
для игр, сыгранных в Праге за период 1980-1989 г.

ВАШЕ РЕШЕНИЕ И МЫСЛИ ТУТ

In [ ]:
from numpy.typing import ArrayLike
from typing import Tuple, Optional, Literal

hypothesis_side = Literal["both", "left", "right"]


class StatTestResult:
    """
    Wrapper that carries information about performed statistical test.

    Attributes
    ----------
    statistics : float
        value of the test's statistics.
    p_value : float
        obtained p-value for the test's statistics.
    significance : float
        significance level for the performed test
    critical_value : float
        critical value of the test's statistics for the given significance level
    test_name : str
        name of the test
    null_name: str
        formulation of the null hypothesis
    alternative_name : str
        formulation of the alternative hypothesis
    verdict:
        test's verdict
    """
    def __init__(self, statistics : float, p_value : float, significance : float, critical_value : float,\
                        test_name : str, null_name : str, alternative_name : str):
        self.statistics = statistics
        self.p_value = p_value
        self.significance = significance
        self.critical_value = critical_value
        self.name = test_name
        self.null = null_name
        self.alternative = alternative_name

        if (p_value < significance):
            self.verdict = f"Reject H0 at significance level {significance}"
        else:
            self.verdict = f"Fail to reject H0 at significance level {significance}"

    def __repr__(self):
        return f"""
        {self.name}.
        H0: {self.null}
        H1: {self.alternative}
        ===================================
        Statistics value: {self.statistics}. Critical value: {self.critical_value}
        P-value: {self.p_value},
        Verdict: {self.verdict}
        """
    def __str__(self):
        return f"Statistics: {self.statistics}, P-value: {self.p_value}"

: 

In [ ]:
def z_test_one_sample(sample : ArrayLike, null_mean : float,
                      significance : float = 0.05, tail : hypothesis_side = 'both') -> StatTestResult:
    """
        Performs asymptotic z-test for the one sample

        Parameters
        ----------
        sample : ArrayLike
            sample on which z-test will be performed
        null_mean : float
            value of mean under null hypothesis.
        significance : float
            significance level for the performed test
        tail : Literal["both", "left", "right"]
            alternative side

        Returns
        -------
        res : StatTestResult
            An object containing infomration about test results
    """
    sample_mean = np.mean(sample)
    sample_std = sample.std()
    n = len(sample)
    se = sample_std / np.sqrt(n)
    z_stat = (sample_mean - null_mean) / se

    if tail == 'both':
        p_value = 2 - 2 * stats.norm.cdf(abs(z_stat))
        critical_value = stats.norm.ppf(1 - significance/2)
        return StatTestResult(z_stat, p_value, significance, (critical_value, -critical_value), 'z-test', f'E[X] = {null_mean}', f'E[X] != {null_mean}')
    elif tail == 'right':
        p_value = 1 - stats.norm.cdf(z_stat)
        critical_value = stats.norm.ppf(1 - significance)
        return StatTestResult(z_stat, p_value, significance, critical_value, 'z-test', f'E[X] <= {null_mean}', f'E[X] > {null_mean}')

    elif tail == 'left':
        p_value = stats.norm.cdf(z_stat)
        critical_value = stats.norm.ppf(significance)
        return StatTestResult(z_stat, p_value, significance, critical_value, 'z-test', f'E[X] >= {null_mean}', f'E[X] < {null_mean}')


: 

In [ ]:
prague_games = games_df[(games_df['site'] == 'Prague') & (games_df['date'].between(1980, 1989))]

data = prague_games['elo_average'].values

: 

In [ ]:
z_test_one_sample(data, null_mean=2389, significance=0.05, tail="both")


: 

### Результаты теста

- **Критические значения**: \($-1.96; 1.96)$ для двустороннего теста при уровне значимости \($\alpha = 0.05)$.
- **P-value**: \($ 0.9598)$ (значительно больше уровня значимости \($ 0.05) $.

- Мы **не отвергаем** нулевую гипотезу \($ H_0)$ , так как наблюдаемая Z-статистика попадает в область принятия, а P-value 0.9598 значительно больше уровня значимости 0.05.




### Назад в будущее. Рейтинг игр в Москве. (3 балла)
После того как вы проверили эту гипотезу, проверьте гипотезу о том, что средний рейтинг классических партий проводимых в Москве в 2010-2019 годах выше, чем рейтинг в 2000-2009 годах. Вспомните как устроен z-тест для двух выборок и
1. сформулируйте нулевую гипотезу и альтернативу;
2. укажите статистику и её распределение при верной нулевой гипотезе;
3. реализуйте тест и проверьте гипотезу на уровне значимости 0.05;

**Гипотезы:**


- **Нулевая гипотеза** $ H_0 $ :
  $\
  \mu_{2010-2019} >= \mu_{2000-2009}
  $
- **Альтернативная гипотеза**  $ H_1 $ : Средний рейтинг в 2010–2019 годах ниже, чем в 2000–2009 годах:
  $
  \mu_{2010-2019} < \mu_{2000-2009}
  $

---

Статистика теста:
Для проверки гипотезы  используем $ z $-тест для двух независимых выборок. Формула $ z $ -статистики:

$
z = \frac{\bar{X}_1 - \bar{X}_2}{\sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}}
$

Где:
- $ \bar{X}_1, \bar{X}_2 $: Средние значения выборок за 2010–2019 и 2000–2009 годы соответственно.
- $ s_1^2, s_2^2 $: Дисперсии выборок.
- $ n_1, n_2 $: Размеры выборок.

---
При верной нулевой гипотезе распределение $ z $-статистики стремится к стандартному нормальному распределению:

$
z \sim N(0, 1)
$


In [ ]:
def z_test_two_sample(sample_x : ArrayLike, sample_y : ArrayLike, null_mean_diff : float = 0,
                      significance : float = 0.05, tail : hypothesis_side = 'both') -> StatTestResult:
    """
        Performs asymptotic z-test for the two samples

        Parameters
        ----------
        sample_x : ArrayLike
            first sample for the z-test
        sample_y : ArrayLike
            second sample for the z-test
        null_mean_diff : float
            differences of means under null hypothesis.
        significance : float
            significance level for the performed test
        tail : Literal["both", "left", "right"]
            alternative side

        Returns
        -------
        res : StatTestResult
            An object containing infomration about test results
    """

    mean_x, mean_y = np.mean(sample_x), np.mean(sample_y)
    std_x, std_y = sample_x.std(), sample_y.std()
    n_x, n_y = len(sample_x), len(sample_y)
    se = np.sqrt((std_x**2 / n_x) + (std_y**2 / n_y))
    z_stat = (mean_x - mean_y - null_mean_diff) / se

    if tail == 'both':
        p_value = 2 * (1 - stats.norm.cdf(abs(z_stat)))
        critical_value = stats.norm.ppf(1 - significance / 2)
        return StatTestResult(z_stat, p_value, significance,
                              (-critical_value, critical_value),
                              'z-test',
                              f'E[X] - E[Y] = {null_mean_diff}',
                              f'E[X] - E[Y] != {null_mean_diff}')
    elif tail == 'right':
        p_value = 1 - stats.norm.cdf(z_stat)
        critical_value = stats.norm.ppf(1 - significance)
        return StatTestResult(z_stat, p_value, significance,
                              critical_value,
                              'z-test',
                              f'E[X] - E[Y] <= {null_mean_diff}',
                              f'E[X] - E[Y] > {null_mean_diff}')
    elif tail == 'left':
        p_value = stats.norm.cdf(z_stat)
        critical_value = stats.norm.ppf(significance)
        return StatTestResult(z_stat, p_value, significance,
                              critical_value,
                              'z-test',
                              f'E[X] - E[Y] >= {null_mean_diff}',
                              f'E[X] - E[Y] < {null_mean_diff}')

: 

In [ ]:
data_x = games_df[(games_df['site'] == 'Moscow') &
                  (games_df['date'].between(2010, 2019)) &
                  (games_df['control'] == 'slow')]['elo_average'].values

data_y = games_df[(games_df['site'] == 'Moscow') &
                  (games_df['date'].between(2000, 2009)) &
                  (games_df['control'] == 'slow')]['elo_average'].values


: 

In [ ]:
z_test_two_sample(data_x, data_y, null_mean_diff=0, significance=0.05, tail="left")

: 

**Результаты теста** :

**P-value**: $ 3.69 \times 10^{-61}$ (крайне мало, гораздо меньше уровня значимости $\alpha = 0.05 $ ).


Мы отвергаем нулевую гипотезу $ H_0 $  на уровне значимости $\alpha = 0.05 $.

На основе полученных данных можно сделать вывод, что средний рейтинг классических партий, проводимых в Москве в 2010–2019 годах, значительно ниже, чем в 2000–2009 годах.


## Задача 1. Инфляция рейтинга (5 баллов)

В течение времени правила подсчета рейтинга менялись. Для сравнения возьмем партии в классическом временном контроле и их рейтинги за периоды 2000-2009 года и 2010-2019 год. Нарисуйте гистограммы, которые описывают распределение усредненных рейтингов оппонентов в партиях за эти периоды.

In [ ]:
slow_2000_2009 = games_df[(games_df['control'] == 'slow') & (games_df['date'].between(2000, 2009))]['elo_average'].values
slow_2010_2019 = games_df[(games_df['control'] == 'slow') & (games_df['date'].between(2010, 2019))]['elo_average'].values

: 

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 8))

axs[0].hist(slow_2000_2009, bins=30, alpha=0.7, color='blue', edgecolor='black')
axs[0].set_title("Распределение усредненного рейтинга в классическом формате (2000-2009 гг.)")
axs[0].set_xlabel('Рейтинг')
axs[0].set_ylabel('Частота')

axs[1].hist(slow_2010_2019, bins=30, alpha=0.7, color='green', edgecolor='black')
axs[1].set_title("Распределение усредненного рейтинга в классическом формате (2010-2019 гг.)")
axs[1].set_xlabel('Рейтинг')
axs[1].set_ylabel('Частота')

plt.tight_layout()
plt.show()

: 

Распространено мнение, что в силу изменений правил подсчета, высокий рейтинг стало получить легче --- произошла инфляция рейтинга. Проверьте, так ли это на самом деле, с помощью рангового критерия [Уилкоксона-Манна-Уитни](http://www.machinelearning.ru/wiki/index.php?title=Критерий_Уилкоксона-Манна-Уитни). В качестве распределения статистики можете брать подходящую аппроксимацию нормальным распределением с поправкой на повторяющиеся значения. Как и ранее,
1. сформулируйте нулевую гипотезу и альтернативу; объясните смысл математической формулировки нулевой гипотезы;
2. укажите статистику и её распределение при верной нулевой гипотезе;
3. реализуйте тест и проверьте гипотезу на уровне значимости 0.05;

**Гипотезы :**



- **Нулевая гипотеза $ H_0 $**: Вероятность того, что рейтинг в период 2010-2019 выше, чем рейтинг в период 2000-2009, не меньше 1/2.
  $
  H_0: P(X > Y) \geq \frac{1}{2}
  $

- **Альтернативная гипотеза $ H_1 $**: Вероятность того, что рейтинг в период 2010-2019 выше, чем рейтинг в период 2000-2009, меньше 1/2.
  $
  H_1: P(X > Y) < \frac{1}{2}
  $

Где $ X $ — рейтинги в период 2010-2019 годов, $ Y $ — рейтинги в период 2000-2009 годов.

Альтернативная гипотеза предполагает, что вероятность  меньше 1/2, что указывает на инфляцию рейтинга.

**Статистики и распределение:**

- **Статистика $ U_X $** для выборки $X$:

  $
  U_X = n_X n_Y + \frac{n_X (n_X + 1)}{2} - R_X,
  $
  где:
  - $ n_X $ и $ n_Y $ — размеры двух выборок.
  - $ R_X $ — сумма рангов значений выборки $ X $.

- **Статистика  $ U_Y $** для выборки $Y$:

  $
  U_Y = n_X n_Y + \frac{n_Y (n_Y + 1)}{2} - R_Y,
  $
  где:
  - $ n_X $ и $ n_Y $ — размеры двух выборок.
  - $ R_Y $ — сумма рангов значений выборки $ Y $.


- **Распределение статистики**:
  При условии верности нулевой гипотезы $ H_0 $, нормализованная и центрированная статистика Манна-Уитни стремится к стандартному нормальному распределению:

  $
  \tilde{U} = \frac{U - \frac{1}{2}mn}{\sqrt{\frac{1}{12}mn(n+m+1)}}
  $

  где $ m = n_X $ и $ n = n_Y $.


In [ ]:
from scipy.stats import rankdata # this can help. See docs.

def mwu_test(sample_x : ArrayLike, sample_y : ArrayLike, significance : float = 0.05, tail : hypothesis_side = 'both') -> StatTestResult:
    """
        Performs Mann–Whitney U-test with correct support for ties in the data.
        REMARK: Test uses normal approximation for the distribution of test statistics
                under null hypothesis.

        Parameters
        ----------
        sample_x : ArrayLike
            first sample for the U-test
        sample_y : ArrayLike
            second sample for the U-test
        significance : float
            significance level for the performed test
        tail : Literal["both", "left", "right"]
            alternative side

        Returns
        -------
        res : StatTestResult
            An object containing infomration about test results
    """

    combined = np.concatenate([sample_x, sample_y])
    ranks = rankdata(combined)

    ranks_x = ranks[:len(sample_x)]
    ranks_y = ranks[len(sample_x):]

    R_x = np.sum(ranks_x)
    R_y = np.sum(ranks_y)

    n_x = len(sample_x)
    n_y = len(sample_y)
    n = n_x + n_y

    U_x = n_x * n_y + (n_x * (n_x + 1)) / 2 - R_x
    U_y = n_x * n_y + (n_y * (n_y + 1)) / 2 - R_y
    U = min(U_x, U_y)

    unique, counts = np.unique(ranks, return_counts=True)
    t_k = np.sum((counts**3 - counts) / (n**2 - n))

    mean_U = n_x * n_y / 2
    std_U = np.sqrt(n_x * n_y * (n + 1 - t_k) / 12)
    z = (U - mean_U) / std_U

    if tail == 'both':
        p_value = 2 * (1 - stats.norm.cdf(abs(z)))
        crit_value = stats.norm.ppf(1 - significance / 2)
        return StatTestResult(z, p_value, significance, crit_value, 'Mann-Whitney U-test',
                              f'P(X > Y) >= 1/2', f'P(X > Y) != 1/2')
    elif tail == 'right':
        p_value = 1 - stats.norm.cdf(z)
        crit_value = stats.norm.ppf(1 - significance)
        return StatTestResult(z, p_value, significance, crit_value, 'Mann-Whitney U-test',
                              f'P(X > Y) >= 1/2', f'P(X > Y) < 1/2')
    elif tail == 'left':
        p_value = stats.norm.cdf(z)
        crit_value = stats.norm.ppf(significance)
        return StatTestResult(z, p_value, significance, crit_value, 'Mann-Whitney U-test',
                              f'P(X > Y) <= 1/2', f'P(X > Y) > 1/2')


: 

In [ ]:
data_2000_2009 = games_df[(games_df['date'] >= 2000) & (games_df['date'] <= 2009) & (games_df['control'] == 'slow')]['elo_average']
data_2010_2019 = games_df[(games_df['date'] >= 2010) & (games_df['date'] <= 2019) & (games_df['control'] == 'slow')]['elo_average']

mwu_test(data_2010_2019, data_2000_2009, significance=0.05, tail="left")


: 

Результаты теста показывают, что средний рейтинг в классических партиях в период 2010-2019 годов действительно выше, чем рейтинг в период 2000-2009 годов, что указывает на наличие инфляции рейтинга.

## Задача 2. Рейтинги и исход партий (7.5 баллов)

Исследуйте как, в зависимости от рейтинга участников, меняется распределение исхода партии. Мы предлагаем вам изучить как
- Разница в рейтинге влияет на шансы на результативный исход (победа одной из сторон)
- С ростом среднего рейтинга участников меняются шансы на победу

### Разрыв рейтинга и шансы на победу (5 баллов)

Нарисуйте гистограммы, показывающее как устроены исходы партий в зависимости от разницы в рейтинге для партий в разных формах временного контролля.

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 16))

slow_games.groupby('result')['elo_difference'].plot.hist(ax=axs[0], alpha=0.5, legend=True, bins=30)
axs[0].set_title("Распределение разницы в рейтинге относительно исхода партии в медленных играх")
axs[0].set_xlabel("Разница в рейтинге")
axs[0].set_ylabel("Частота")

rapid_games.groupby('result')['elo_difference'].plot.hist(ax=axs[1], alpha=0.5, legend=True, bins=30)
axs[1].set_title("Распределение разницы в рейтинге относительно исхода партии в рапид играх")
axs[1].set_xlabel("Разница в рейтинге")
axs[1].set_ylabel("Частота")

blitz_games.groupby('result')['elo_difference'].plot.hist(ax=axs[2], alpha=0.5, legend=True, bins=30)
axs[2].set_title("Распределение разницы в рейтинге относительно исхода партии в блиц играх")
axs[2].set_xlabel("Разница в рейтинге")
axs[2].set_ylabel("Частота")

plt.tight_layout()
plt.show()




: 

Проверьте гипотезу "симметрии":
> Правда ли что шансы победы белых при разрыве рейтинга между участниками в $X$ пунктов такие же, как шансы на победу чёрных при разрыве рейтинга в $-X$ пунктов?

Для проверки такой гипотезы будем использовать критерий $\chi^2$ для проверки независимости факторов.
- Подумайте, как должна выглядеть факторная таблица в этом случае?
- Сформулируйте как интерпретировать нулевую гипотезу критерия $\chi^2$ в контексте проверяемой нами гипотезы.
- Реализуйте проверку гипотезы.

**Факторная таблица:**

Столбцы представляют исходы партии: победа белых и победа черных.
Строки представляют величину разрыва в рейтинге: положительное значение разрыва (X, когда белые имеют преимущество) и отрицательное значение разрыва (-X, когда черные имеют преимущество)

### Интерпретация гипотезы с критерием $ \chi^2\ $:

Мы проверяем независимость факторов: исхода партии (победа белых/черных) и разрыва в рейтинге.  
Если критерий $\chi^2\$ даст p-value меньше уровня значимости = 0.05, это означает, что между этими факторами существует зависимость, и мы отвергаем нулевую гипотезу о симметрии.


In [ ]:
def chi2_contingency(observed : ArrayLike, significance : float = 0.05) -> StatTestResult:
    """
        Performs chi-square independence test test for contingency table

        Parameters
        ----------
        observed : ArrayLike
            contingency tablefor the test
        significance : float
            significance level for the performed test

        Returns
        -------
        res : StatTestResult
            An object containing infomration about test results
    """
    observed = np.array(observed)

    n = np.sum(observed)

    num_rows, num_cols = observed.shape
    dof = (num_rows - 1) * (num_cols - 1)

    row_sums = np.sum(observed, axis=1).reshape(-1, 1)
    col_sums = np.sum(observed, axis=0).reshape(1, -1)
    expected = (row_sums @ col_sums) / n

    chi2_stat = np.sum(((observed - expected) ** 2) / expected)
    p_value = 1 - stats.chi2.cdf(chi2_stat, dof)
    critical_value = stats.chi2.ppf(1 - significance, dof)

    null_name = "The outcome of the game is independent of the elo difference"
    alternative_name = "The outcome of the game depends on the elo difference"

    return StatTestResult(
        statistics=chi2_stat,
        p_value=p_value,
        significance=significance,
        critical_value=critical_value,
        test_name="Chi-Squared Independence Test",
        null_name=null_name,
        alternative_name=alternative_name
    )




: 

Проверьте сформулированную выше гипотезу для ваших данных:

In [ ]:
agg_white = games_df[games_df['result'] == '1-0'].groupby('elo_difference').size().reset_index(name='white_wins')
agg_black = games_df[games_df['result'] == '0-1'].groupby('elo_difference').size().reset_index(name='black_wins')
agg_black['elo_difference'] = -agg_black['elo_difference']
merged_table = pd.merge(agg_white, agg_black, on='elo_difference', how='outer').fillna(0)

: 

In [ ]:
merged_table.drop(columns=['elo_difference'], inplace=True)
merged_table

: 

In [ ]:
chi2_contingency(merged_table.to_numpy(), significance=0.05)

: 

**Заключение**: получилось, что гипотеза неверна, симметрия не наблюдается

### Средний рейтинг и распределение исходов (2.5 балла)
Существует распространеное мнение, что чем выше рейтинг участников партии, тем более вероятно, что итогом партии станет ничья. Постройте гистограммы показывающее как устроены исходы партий в зависимости от рейтинга партии (``elo_average``) для партий в разных формах временного контролля.

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 16))

axs[0].set_title("Распределение среднего рейтинга относительно результата в медленных партиях")
sns.histplot(data=slow_games, x='elo_average', hue='result', multiple='stack', ax=axs[0])
axs[0].set_xlabel("Средний рейтинг участников (elo_average)")
axs[0].set_ylabel("Частота")

axs[1].set_title("Распределение среднего рейтинга относительно результата в рапид партиях")
sns.histplot(data=rapid_games, x='elo_average', hue='result', multiple='stack', ax=axs[1])
axs[1].set_xlabel("Средний рейтинг участников (elo_average)")
axs[1].set_ylabel("Частота")

axs[2].set_title("Распределение среднего рейтинга относительно результата в блиц партиях")
sns.histplot(data=blitz_games, x='elo_average', hue='result', multiple='stack', ax=axs[2])
axs[2].set_xlabel("Средний рейтинг участников (elo_average)")
axs[2].set_ylabel("Частота")

plt.tight_layout()
plt.show()

: 

Согласно ChessBase, игроков можно условно делить на следующие категории, в зависимости от их рейтинга:
- 0-1000 Begginer
- 1000-1600 Average club player level
- 1600-2100 Strong club player level
- 2100-2300 International league player
- 2300-2450 International Master (IM) level
- 2450-2650 Grandmaster (GM) level
- \> 2650 Supergrandmaster, world champion level

Вместо игроков, мы будем брать усредненный рейтинг оппонентов в партии и применять классификацию выше. С помощью статистических методов, проверьте влияет ли класс партии на распределение исходов. Используйте уже знакомый вам тест $\chi^2$.
- Подумайте, как должна выглядеть факторная таблица в этом случае?
- Сформулируйте как интерпретировать нулевую гипотезу критерия $\chi^2$ в контексте проверяемой нами гипотезы.
- Реализуйте проверку гипотезы.

**Факторная таблица:**

Столбцы: Исходы партий (Победа белых, Победа черных, Ничья)

Строки: Категории участников по рейтингу

**Нулевая гипотеза:** Класс рейтинга участников не влияет на распределение исходов партий.
То есть, распределение результатов (победа белых, победа черных, ничья) одинаково для всех классов рейтингов.

**Альтернативная гипотеза :** Класс рейтинга участников влияет на распределение исходов партий.

Если критерий даст p-value меньше уровня значимости, это будет свидетельствовать о том, что существует зависимость между классом участников и распределением исходов партий.

In [ ]:
bins   = [0, 1000, 1600, 2100, 2300, 2450, 2650, 3000]
labels = ['Begginer', 'Average', 'Strong', 'International', 'IM', 'GM', 'SuperGM']

games_df_2 = pd.cut(games_df['elo_average'], bins=bins)

: 

In [ ]:
games_df_2

: 

In [ ]:
bins   = [0, 1000, 1600, 2100, 2300, 2450, 2650, 3000]
labels = ['Begginer', 'Average', 'Strong', 'International', 'IM', 'GM', 'SuperGM']

games_df['rating_category'] = pd.cut(games_df['elo_average'], bins=bins, labels=labels)
blitz_games_2 = games_df[games_df['control'] == 'blitz']
rapid_games_2 = games_df[games_df['control'] == 'rapid']
slow_games_2 = games_df[games_df['control'] == 'slow']

table_blitz = pd.crosstab(blitz_games_2['rating_category'], blitz_games_2['result'])
table_rapid = pd.crosstab(rapid_games_2['rating_category'], rapid_games_2['result'])
table_slow = pd.crosstab(slow_games_2['rating_category'], slow_games_2['result'])


: 

In [ ]:
table_slow

: 

In [ ]:
table_rapid

: 

In [ ]:
table_blitz

: 

In [ ]:
table_blitz = pd.crosstab(blitz_games_2['rating_category'], blitz_games_2['result'])
table_blitz = table_blitz.reset_index()

table_rapid = pd.crosstab(rapid_games_2['rating_category'], rapid_games_2['result'])
table_rapid = table_rapid.reset_index()

table_slow = pd.crosstab(slow_games_2['rating_category'], slow_games_2['result'])
table_slow = table_slow.reset_index()

: 

In [ ]:
table_slow

: 

In [ ]:
print('Testing ... for slow format')
chi2_contingency(table_slow.drop('rating_category', axis=1))

: 

In [ ]:
print('Testing ... for rapid format')
chi2_contingency(table_rapid.drop('rating_category', axis=1))

: 

In [ ]:
print('Testing ... for blitz format')
chi2_contingency(table_blitz.drop('rating_category', axis=1))

: 

**Заключение**: существует зависимость между классом участников и распределением исходов партий

Проблема в том что критерий $\chi^2$ позволяет нам проверять факторы на зависимость/независимость, но ничего не говорит о структуре этой зависимости. Реализуйте вычисление ранговой корреляции Спирмена и посчитайте её, чтобы оценить есть ли
- Монотонная зависимость между *шансами на ничью* и *рангом партии*
- Монотонная зависимость между *разницей шансов на победу у белых и у чёрных* и *рангом партии*

Используйте для этого партии сыгранные в блиц-контроле и дайте интерпретацию полученным результатам.

In [ ]:
table_blitz['total_games'] = table_blitz['1-0'] + table_blitz['0-1'] + table_blitz['1/2-1/2']
table_blitz['draw_prob'] = table_blitz['1/2-1/2'] / table_blitz['total_games']
table_blitz['white_win_prob'] = table_blitz['1-0'] / table_blitz['total_games']
table_blitz['black_win_prob'] = table_blitz['0-1'] / table_blitz['total_games']
table_blitz['dominance'] = table_blitz['white_win_prob'] - table_blitz['black_win_prob']

: 

In [ ]:
def spearman_rank_correlation(observations : ArrayLike) -> float:
    """
        Calculates spearmean's rank correlation coefficient for data in the format
            (1, X[1]), (2, X[2]), .... (n, X[n])
        i.e. for the paired rank data sorted by the first component. No ties assumed
        in the data X.

        Parameters
        ----------
        observed : ArrayLike
             array of values X[1], X[2], ... X[n],

        Returns
        -------
        spearman : float
            value of the spearman correlation for the given data
    """
    R = rankdata(observations)
    n = len(observations)
    S = np.arange(1, n + 1)
    spearman_corr = np.corrcoef(R, S)[0, 1]

    return spearman_corr

: 

In [ ]:
rho = spearman_rank_correlation(table_blitz['draw_prob'])
print(f'Correlation between draw odds and game rank: {rho}')

: 

In [ ]:
rho = spearman_rank_correlation(table_blitz['dominance'])
print(f'Correlation between white dominance over black and game rank: {rho}')

: 

**Заключение:** Значение корреляции равно 1.0, что говорит о сильной положительной линейной зависимости между вероятностью ничьей и рангом партии.
Это означает, что с ростом ранга партии (то есть, когда участники партии являются более сильными игроками), вероятность ничьей увеличивается

Значение корреляции равно 0.214, что говорит о слабой положительной зависимости между доминированием белых над черными и рангом партии.
Это значение корреляции указывает, что с увеличением уровня рейтинга участников, шансы на победу белых над черными немного увеличиваются. Однако эта связь не является сильной.

## Задача 3. Статистика дебютов (2.5 балла)

Другим распространённым мнением является то что выбор дебюта на высоком уровне не влияет на результативность партии. Все дебюты, согласно энциклопедии шахматных дебютов (ECO) делятся на 5 категорий
- A: Фланговые варианты.
- B: Полуоткрыте варианты.
- C: Открытые варианты.
- D: Закрытые и полузакрытые варианты.
- E: Системы типа индийской защиты.

Так как у нас нет доступа к партиям ИИ, которые сейчас считаются эталоном точной игры, мы ограничимся данными по партиям уровня супергроссмейстеров.
Используя статистические методы, проверьте:
- Есть ли разница в предпочтениях дебютов (с точки зрения приведенных выше категорий) между партиями уровня супергроссмейстеров и партиями уровня гроссмейстеров в классическом временном контроле?
- Правда ли что выбор между открытым (C) и закрытым (D) началом не влияет на распределение исхода партии? Проверьте это для разных форм временного контроля. Используйте партии только уровня супергроссмейстеров.

Для проверки будем использовать критерий $\chi^2$ (снова). Для каждой гипотезы, которое вы собираетесь проверять:
- сделайте визуализацию соответствующих данных;
- сформулируйте нулевую и альтернативную гипотезу. Опишите как применить критерий $\chi^2$ для проверки этих гипотез;
- реализуйте проверку гипотезы и дайте интерпретацию результатам;

ВАШЕ РЕШЕНИЕ И МЫСЛИ ТУТ

In [ ]:
supergm_slow_games = games_df[(games_df['rating_category'] == 'SuperGM') & (games_df['control'] == 'slow')]

opening_distribution = supergm_slow_games['opening'].value_counts().reset_index()
opening_distribution.columns = ['opening', 'count']

plt.figure(figsize=(5, 4))
plt.bar(opening_distribution['opening'], opening_distribution['count'], color='skyblue')
plt.xlabel('Opening Category')
plt.ylabel('Number of Games')
plt.title('Distribution of Openings for SuperGMs in Slow Format')
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()

: 

In [ ]:
gm_slow_games = games_df[(games_df['rating_category'] == 'GM') & (games_df['control'] == 'slow')]

opening_distribution_gm = gm_slow_games['opening'].value_counts().reset_index()
opening_distribution_gm.columns = ['opening', 'count']

plt.figure(figsize=(5, 4))
plt.bar(opening_distribution_gm['opening'], opening_distribution_gm['count'], color='lightcoral')
plt.xlabel('Opening Category')
plt.ylabel('Number of Games')
plt.title('Distribution of Openings for GMs in Slow Format')
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()

: 

**Нулевая гипотеза:**

Нет разницы в предпочтениях дебютов между партиями уровня супергроссмейстеров и партиями уровня гроссмейстеров в классическом временном контроле. То есть, распределение дебютов одинаково для двух категорий игроков.

**Альтернативная гипотеза:**

Есть разница в предпочтениях дебютов между партиями уровня супергроссмейстеров и партиями уровня гроссмейстеров в классическом временном контроле. Это значит, что частота выбора дебютов между уровнями игроков отличается.

Для проверки нулевой гипотезы сравниваем значение
𝜒^2 с критическим значением, соответствующим  уровню значимости и степеням свободы.
Если рассчитанное значение больше критического, то нулевая гипотеза отвергается в пользу альтернативной гипотезы

In [ ]:
contingency_table = pd.crosstab(games_df['rating_category'], games_df['opening'])
contingency_table = contingency_table.loc[['GM', 'SuperGM']]

: 

In [ ]:
contingency_table

: 

In [ ]:
chi2_contingency(contingency_table)

: 

**Заключение**: Получилось, что есть разница в выборе дебютов между партиями уровня супергроссмейстеров и партиями уровня гроссмейстеров в классическом временном контроле

Теперь проверьте влияет ли выбор между открытым и закрытым началом:

In [ ]:
super_gm_games = games_df[games_df['elo_average'] >= 2650]
super_gm_games_c = super_gm_games[super_gm_games['opening'] == 'C']
super_gm_games_d = super_gm_games[super_gm_games['opening'] == 'D']

super_gm_openings_cd = pd.concat([super_gm_games_c, super_gm_games_d])

slow_games_3 = super_gm_openings_cd[super_gm_openings_cd['control'] == 'slow']
rapid_games_3 = super_gm_openings_cd[super_gm_openings_cd['control'] == 'rapid']
blitz_games_3 = super_gm_openings_cd[super_gm_openings_cd['control'] == 'blitz']

plt.figure(figsize=(10, 5))


plt.title('Distribution of results by openings (C vs D) for SuperGMs in slow format')
sns.countplot(data=slow_games_3, x='opening', hue='result')
plt.xlabel('Opening Category (C: Open, D: Closed)')
plt.ylabel('Number of Games')
plt.legend(title='Result')
plt.show()


plt.figure(figsize=(10, 5))
plt.title('Distribution of results by openings (C vs D) for SuperGMs in rapid format')
sns.countplot(data=rapid_games_3, x='opening', hue='result')
plt.xlabel('Opening Category (C: Open, D: Closed)')
plt.ylabel('Number of Games')
plt.legend(title='Result')
plt.show()


plt.figure(figsize=(10, 5))
plt.title('Distribution of results by openings (C vs D) for SuperGMs in blitz format')
sns.countplot(data=blitz_games_3, x='opening', hue='result')
plt.xlabel('Opening Category (C: Open, D: Closed)')
plt.ylabel('Number of Games')
plt.legend(title='Result')
plt.show()

: 

**Нулевая гипотеза:**

Выбор между открытым и закрытым дебютом независим от результата партии.

**Альтернативная гипотеза:**

Выбор между открытым и закрытым дебютом влияет на результат партии.


Сделаем три таблицы по дебюту открытых и закрытых категорий и по результату для трех временных контролей

In [ ]:
super_gm_games = games_df[games_df['rating_category'] == 'SuperGM']

super_gm_openings_cd = super_gm_games[super_gm_games['opening'].isin(['C', 'D'])]

slow_games = super_gm_openings_cd[super_gm_openings_cd['control'] == 'slow']
rapid_games = super_gm_openings_cd[super_gm_openings_cd['control'] == 'rapid']
blitz_games = super_gm_openings_cd[super_gm_openings_cd['control'] == 'blitz']

table_slow = pd.crosstab(slow_games['opening'], slow_games['result'])
table_rapid = pd.crosstab(rapid_games['opening'], rapid_games['result'])
table_blitz = pd.crosstab(blitz_games['opening'], blitz_games['result'])


: 

In [ ]:
table_slow

: 

In [ ]:
table_rapid

: 

In [ ]:
table_blitz

: 

In [ ]:
chi2_contingency(table_blitz)

: 

In [ ]:
chi2_contingency(table_rapid)

: 

In [ ]:
chi2_contingency(table_slow)

: 

Что изменится если брать уровень значимости 0.01? А если брать уровень значимости 0.1? Дайте ответы на эти вопросы и напишите интерпретацию полученных результатов.

**Заключение**: На уровне значимости 0.05 можно утверждать, что только в классическом контроле (slow) разница в рейтинге участников влияет на результат партии. Вообще,можно увидеть, что чем меньше времени, тем больше влияние дебюта.

На уровне значимости 0,01 зависимость не подтверждается.
На уровне значимости 0,1 зависимость подтверждается для рапид и классического контроля.

## Задача 4. Исследование длин партий в разных формах временного контроля (10 баллов)

Попробуйте разобраться в том, какому распределению следуют длительности партий в формате рапид. Выберете несколько распределений и попробуйте оценить их параметры. Постройте гистограммы на которых изображено распределение длительности партий и теоретические плотности распределений с оценёнными параметрами.

Для оценки параметров распределений можно использовать метод ``fit()`` класса ``scipy.rv_continious()``.

In [ ]:
length_counts = rapid_games_2.groupby('length').size().reset_index(name='count')

plt.figure(figsize=(20, 10))

sns.barplot(data=length_counts, x='length', y='count', palette='Blues', width=1)

plt.xticks(plt.xticks()[0][::25])

plt.xlabel('Length of Game')
plt.ylabel('Count of Games')
plt.title('Distribution of Game Length in Rapid Format')

plt.show()

: 

In [ ]:
from scipy.stats import gamma, lognorm, exponnorm, invgauss, invgamma, rayleigh, maxwell
data = rapid_games['length'].to_numpy().flatten()
fig, axs = plt.subplots(7, 1, figsize=(15, 35))
probs = np.bincount(data) / len(data)
probs_x = np.arange(len(probs))
x = np.linspace(data.min(), data.max(), 1000)
distributions = [gamma, lognorm, exponnorm, invgauss, invgamma, rayleigh, maxwell]

for dist, ax in zip(distributions, axs):
    params = dist.fit(data)

    *params, loc, scale = params

    ax.bar(probs_x, probs, width=1, alpha=.5)

    ax.plot(x, dist.pdf(x, *params, loc, scale), color='red', linewidth=2)

    ax.set_title(f'Fitted {dist.name.capitalize()} Distribution vs Real Data')
    ax.set_xlabel('Length of Game')
    ax.set_ylabel('Count/Frequency')

plt.tight_layout()
plt.show()

: 

Выберите три наиболее на ваш взгляд подходящих распределения и постройте для них Q-Q график. Подробнее про Q-Q графики можно прочитать [здесь](https://habr.com/ru/articles/578754/) (обратите внимание, что в статье делается упор на Q-Q графики относительно нормального распределения --- вам же нужно построить графики относительно распределений, параметры которых вы оценили).

In [ ]:
def qqplot(data : ArrayLike, distribution: stats.rv_continuous,  quantiles : int,  ax : plt.axis) -> plt.axis:
    """
        Plots Q-Q plot agains theoretical distribution.

        Parameters
        ----------
        data : ArrayLike
            sample data
        distribution: stats.rv_continuous
            theoretical distribution agains which quantiles will be plotted
        quantiles: int
            number of quantiles (must be less than size of the data)
        ax: plt.axis
            PyPlot Axis object, on which QQ-plot should be plotted

        Returns
        -------
        ax: plt.axis
            PyPlot Axis object with QQ-plot
    """

    quantiles_vals = np.linspace(0, 1, quantiles + 2)[1:-1]
    dist_quantiles = distribution.ppf(quantiles_vals, *distribution.fit(data))
    sample_quantiles = np.quantile(data, quantiles_vals)
    ax.scatter(dist_quantiles, sample_quantiles, color='blue')
    ax.plot([dist_quantiles.min(), dist_quantiles.max()],
            [sample_quantiles.min(), sample_quantiles.max()],
            linestyle='--', color='red')

    ax.set_title(f"Q-Q Plot vs {distribution.name}")
    ax.set_xlabel("Theoretical Quantiles")
    ax.set_ylabel("Sample Quantiles")

    return ax

: 

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

data = np.random.exponential(scale=1, size=1000)

qqplot(data, stats.lognorm, quantiles=50, ax=axs[0])
qqplot(data, stats.exponnorm, quantiles=50, ax=axs[1])
qqplot(data, stats.invgamma, quantiles=50, ax=axs[2])

plt.tight_layout()
plt.show()

: 

 На основе всего выше сделанного выберете одно распределение, которое вам кажется наиболее подходящим. После того как вы выбрали распределение $F_0$, проверьте соответствующую гипотезу согласия. Конкретнее, проверьте гипотезу
$$
H_0: X_1,..,X_n \sim F_0
$$
против альтернативы
$$
H_A: X_1,..,X_n \nsim F_0
$$


Такую гипотезу часто проверяют с помощью статистики критерия Колмогорова-Смирнова:
$$
D = \sqrt{n} \sup_x \vert F_{0} - \hat{F}(x)\vert,
$$
где $\hat{F}$ -- эмпирическая функция распределения, которая задётся как
$$
\hat{F}(x) = \frac{1}{n}\sum_{i=1}^n \mathbb{1}(X_i \leq x).
$$
Выражение $1(условие)$ равно $1$, если условие верно и $0$ в противном случае -- так выше считается количество элементов выборки $\leq x$. Статистика $D$ при больших $n$ (сотен уже достаточно) имеет распределение Колмогорова, так что можно построить критерий для проверки. Чтобы упростить техническую часть с подсчётом $D$, воспользуйтесь готовым тестом из пакета scipy.stats.

In [ ]:
dist = stats.lognorm
params = dist.fit(rapid_games['length'])
stats.kstest(rapid_games['length'], dist.cdf, args=params)

: 

Объясните, почему получилось такое значение несмотря на то, что подобранное вами распределение достаточно неплохо описывает данные? Подумайте, можно ли как-то изменить процедуру проверки для того чтобы можно было воспользоваться этим тестом? Обратите внимание на две вещи:
1. Как считается статистка критерия КС и почему для наших данных эмпирическая функция распределения никогда не будет сходится к функции распределения выбранного вами распределения? Как можно сделать дискретные данные  не-дискретными?
2.  Обратите внимание на выброс в данных в районе где достигается значение статистики критерия (statistic location). С чем может быть связан этот выброс и как можно было бы его устранить?

Опишите и реализуйте измененную процедуру.

Тест Колмогорова-Смирнова чувствителен к дискретности данных. Наши данные в длинах партий имеют дискретные значения, а распределение, которое мы используем  для проверки, является непрерывным.

Из-за дискретности данных эмпирическая функция распределения имеет резкие ступенчатые изменения, а кумулятивная функция непрерывного распределения изменяется плавно. Поэтому есть значительные различия между эмпирической и теоретической функциями в некоторых точках, что увеличивает статистику.


Также видно, что статистика достигает максимума в определенной области (statistic_location=53).
Этот выброс может быть связан с некоторыми аномальными значениями, которые значительно отличаются от остальной части распределения. Такие выбросы могут повлиять на результат теста.

Можно добавить небольшое случайное "шумовое" значение к длинам партий, чтобы сделать данные менее дискретными и ближе к непрерывному распределению. Это поможет эмпирической функции распределения более плавно приближаться к теоретической функции распределения.
Перед выполнением теста можно удалить выбросы.

In [ ]:
rapid_games['length_noisy'] = rapid_games['length'] + np.random.normal(0, 0.5, size=len(rapid_games))

Q1 = np.percentile(rapid_games['length_noisy'], 25)
Q3 = np.percentile(rapid_games['length_noisy'], 75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

filtered_data = rapid_games[(rapid_games['length_noisy'] >= lower_bound) & (rapid_games['length_noisy'] <= upper_bound)]['length_noisy']

: 

In [ ]:
params = stats.lognorm.fit(filtered_data)

stats.kstest(filtered_data, stats.lognorm.cdf, args=params)

: 